In [254]:
import os
import pandas as pd 

In [255]:
def read_csv_file(filename=""):
    df_raw = pd.read_csv(filename)
    return df_raw

data_list = []
f_name = "../../../JupyterNotebooks/Comparison_Multivariate/dataset/DivvyBikes/raw/2021{}-divvy-tripdata.csv"
for i in ("01","02","03","04","05","06","08","09","10","11","12"):
#for i in ("08","09","10"):
    data_list.append(read_csv_file (f_name.format(i)))

In [256]:
df_raw = pd.concat(data_list)

In [257]:
df_raw['started_at'] = pd.to_datetime(df_raw['started_at'])
df_raw.set_index('started_at', inplace=True)


In [258]:
checkouts_per_hour = df_raw.groupby('start_station_id').resample('1H').size()
checkouts_per_hour = checkouts_per_hour.reset_index(name='number_of_checkouts')
pivot_df = checkouts_per_hour.pivot(index='started_at', columns='start_station_id', values='number_of_checkouts')



In [259]:
pivot_df.fillna(0).to_csv("../../../JupyterNotebooks/Comparison_Multivariate/dataset/DivvyBikes/raw/2021-12-months-divvy-hourly-tripdata.csv")

In [265]:
pivot_df = pivot_df.fillna(0)
cols = pivot_df.columns
#cols = [i+"_checkout" for i in cols]

In [267]:
cols_dict = {}
for i in cols: 
    cols_dict[i] = i+"_checkout"


In [269]:
pivot_df = pivot_df.rename(columns=cols_dict)

In [270]:
def sum_nextday_checkout(station_id):
    column_filled = pivot_df.fillna(0)[station_id]
    shifted_rolling_sums = column_filled.rolling(window=24, min_periods=1).sum().shift(-24)
    
    ## this has to start from 2, otherwise the last entry will have the sum of all the element in datasets
    for i in range(2, 25):
      shifted_rolling_sums.iloc[-i] = column_filled.iloc[-i+1:].sum() 
    shifted_rolling_sums = shifted_rolling_sums.fillna(0)
    return (shifted_rolling_sums)


In [271]:
for ic in pivot_df.columns:
    ic_new = ic.replace("_checkout","_next1daycheckout")
    pivot_df[ic_new] = sum_nextday_checkout(ic)

In [272]:
pivot_df["13022_next1daycheckout"]

started_at
2021-01-01 00:00:00    7.0
2021-01-01 01:00:00    7.0
2021-01-01 02:00:00    7.0
2021-01-01 03:00:00    7.0
2021-01-01 04:00:00    7.0
                      ... 
2021-12-31 19:00:00    6.0
2021-12-31 20:00:00    4.0
2021-12-31 21:00:00    3.0
2021-12-31 22:00:00    1.0
2021-12-31 23:00:00    0.0
Name: 13022_next1daycheckout, Length: 8760, dtype: float64

In [277]:
pivot_df.rename_axis('date', inplace=True)

In [278]:
pivot_df.fillna(0)[:-24].to_csv("DivvyBikes_hourly_TS_daily_data.csv")

In [279]:
!head DivvyBikes_hourly_TS_daily_data.csv

date,13001_checkout,13006_checkout,13008_checkout,13011_checkout,13016_checkout,13017_checkout,13021_checkout,13022_checkout,13028_checkout,13029_checkout,13033_checkout,13034_checkout,13036_checkout,13037_checkout,13042_checkout,13045_checkout,13050_checkout,13053_checkout,13056_checkout,13058_checkout,13059_checkout,13061_checkout,13063_checkout,13068_checkout,13071_checkout,13073_checkout,13074_checkout,13075_checkout,13080_checkout,13081_checkout,13083_checkout,13084_checkout,13085_checkout,13089_checkout,13091_checkout,13096_checkout,13099_checkout,13102_checkout,13108_checkout,13109_checkout,13124_checkout,13128_checkout,13132_checkout,13133_checkout,13134_checkout,13135_checkout,13136_checkout,13137_checkout,13138_checkout,13139_checkout,13143_checkout,13144_checkout,13146_checkout,13150_checkout,13154_checkout,13155_checkout,13156_checkout,13157_checkout,13158_checkout,13160_checkout,13162_checkout,13163_checkout,13164_checkout,13165_checkout,13179_checkout,13191_checkout,13192

### Code for daily data 
- Following code use the daily data for hourly timestamps, i.e. each of the hourly timestamp will have the number of checkouts for next 24 hours. This will help understand the need of a station for next 24 hours. 
- The model will capture the weekly, monthly seasonality. 


In [309]:
#mdata = pd.read_csv('dataset/DivvyBikes/df2021-8-9-10.csv')
mdata = pd.read_csv('DivvyBikes_hourly_TS_daily_data.csv')
n_cols = 1667
select_nextday = [ i for i in mdata.columns if ("_next1daycheckout" in i)][:200]  # select only first 200 stations 
mdata = mdata[select_nextday]
selected_cols_dict = {}
for i in select_nextday: 
    selected_cols_dict[i] = i.replace("_next1daycheckout", "_checkout")
mdata.rename_axis(selected_cols_dict)
#mdata.to_csv("dataset/DivvyBikes/df2021-8-9-10_VAR_nextdayAshrdata.csv")
mdata.to_csv("df2021-8-9-10_VAR_nextdayAshrdata.csv", index=False)

In [304]:
!head df2021-8-9-10_VAR_nextdayAshrdata.csv

13001_next1daycheckout,13006_next1daycheckout,13008_next1daycheckout,13011_next1daycheckout,13016_next1daycheckout,13017_next1daycheckout,13021_next1daycheckout,13022_next1daycheckout,13028_next1daycheckout,13029_next1daycheckout,13033_next1daycheckout,13034_next1daycheckout,13036_next1daycheckout,13037_next1daycheckout,13042_next1daycheckout,13045_next1daycheckout,13050_next1daycheckout,13053_next1daycheckout,13056_next1daycheckout,13058_next1daycheckout,13059_next1daycheckout,13061_next1daycheckout,13063_next1daycheckout,13068_next1daycheckout,13071_next1daycheckout,13073_next1daycheckout,13074_next1daycheckout,13075_next1daycheckout,13080_next1daycheckout,13081_next1daycheckout,13083_next1daycheckout,13084_next1daycheckout,13085_next1daycheckout,13089_next1daycheckout,13091_next1daycheckout,13096_next1daycheckout,13099_next1daycheckout,13102_next1daycheckout,13108_next1daycheckout,13109_next1daycheckout,13124_next1daycheckout,13128_next1daycheckout,13132_next1daycheckout,13133_next1